In [1]:
import pandas as pd
import numpy as np

import yfinance as yf


In [2]:


#Function that takes a dataframe and weights, and outputs a dataframe with updated ticker symbols, so that it can be used with yfinance.
#It also adds a column called weighted score, which can be used to adjust how you want to weigh enviroment, governance and social score

def ESG_SCORE_WEIGHT(data: pd.DataFrame, weights: np.array):
    if np.sum(weights) != 1:
        return("Weights must sum to 1")
    else:
        data["weighted_score"] = (data["environment_score"] * (3*weights[0])) + \
                           (data["governance_score"] * (3*weights[1])) + \
                           (data["social_score"] * (3*weights[2]))
        for index, row in data.iterrows():
            if row['CurrencyCode'] == 'SEK':
                data.at[index, 'stock_symbol'] += '.ST'
            elif row['CurrencyCode'] == 'NOK':
                data.at[index, 'stock_symbol'] += '.OL'
            elif row['CurrencyCode'] == 'DKK':
                data.at[index, 'stock_symbol'] += '.CO'
        return data
    
    



In [3]:
df1 = ESG_SCORE_WEIGHT(pd.read_excel(r'C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\ESG_DATA1.xlsx'),np.array([1/3,1/3,1/3]))

In [4]:
df1

,Isin,CurrencyCode,company_name,environment_grade,environment_level,environment_score,governance_grade,governance_level,governance_score,social_grade,social_level,social_score,stock_symbol,total,total_grade,total_level,weighted_score
0,SE0011337708,SEK,AAK AB (publ.),AA,Excellent,647,BB,Medium,345,BB,Medium,307,AAK.ST,1299,A,High,1299.0
1,CH0012221716,SEK,ABB Ltd,BB,Medium,306,B,Medium,220,BBB,High,425,ABB.ST,951,BBB,High,951.0
2,SE0007897079,SEK,ACADIA Pharmaceuticals Inc.,B,Medium,290,BB,Medium,300,BB,Medium,344,ACAD.ST,934,BBB,High,934.0
3,SE0014781795,SEK,Addtech AB (publ.),A,High,500,BB,Medium,300,BB,Medium,300,ADDT-B.ST,1100,BBB,High,1100.0
4,CA00830W1059,SEK,AEC Securities Public Company Limited,B,Medium,240,BB,Medium,300,BB,Medium,300,AEC.ST,840,BB,Medium,840.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,DK0060036564,DKK,Spar Nord Bank A/S,B,Medium,230,BB,Medium,300,B,Medium,216,SPNO.CO,746,B,Medium,746.0
283,DK0060696300,DKK,Straker Translations Limited,BBB,High,410,BB,Medium,315,BB,Medium,301,STG.CO,1026,BBB,High,1026.0
284,DK0010311471,DKK,Sydbank A/S,A,High,500,BB,Medium,300,BB,Medium,301,SYDB.CO,1101,BBB,High,1101.0
285,DK0060477503,DKK,Thorney Opportunities Ltd,B,Medium,215,BB,Medium,305,B,Medium,203,TOP.CO,723,B,Medium,723.0


In [19]:
import yfinance as yf
import pandas as pd




#Function that takes a dataframe with the layout as above as well as a timeframe in which you want your data from 

def monthly_close(data: pd.DataFrame, dates: tuple):
    symbols = data['stock_symbol'].unique()

# create a new dataframe to store the monthly closing data
    monthly_close_df = pd.DataFrame()
    for symbol in symbols:
    # retrieve data from yfinance
        stock_data = yf.download(symbol, start=dates[0], end=dates[1], interval='1mo', progress=False)
        
        # extract the 'Close' column and rename it with the stock symbol
        stock_data = stock_data[['Close']].rename(columns={'Close': symbol})
        
        # add the weighted score for the stock
        weighted_score = data.loc[data['stock_symbol']==symbol, 'weighted_score'].iloc[0]
        stock_data[symbol + '_weighted'] = weighted_score
        
        # append the stock data to the monthly_close_df
        monthly_close_df = pd.concat([monthly_close_df, stock_data], axis=1)
    return(monthly_close_df.dropna(axis=1,how='all'))    
        



In [20]:
df2 = monthly_close(df1,dates=('2020-01-01','2022-03-01'))


1 Failed download:
- COLL.ST: Data doesn't exist for startDate = 1577833200, endDate = 1646089200

1 Failed download:
- JOBS.ST: Data doesn't exist for startDate = 1577833200, endDate = 1646089200

1 Failed download:
- MSEIS.OL: No timezone found, symbol may be delisted

1 Failed download:
- SDRL.OL: Data doesn't exist for startDate = 1577833200, endDate = 1646089200


.to_csv('cleaned_data.csv', index=False)

In [21]:
print(df2)


                               AAK.ST  AAK.ST_weighted      ABB.ST  \
Date                                                                 
2019-12-31 23:00:00+00:00  181.050003           1299.0  224.800003   
2020-01-31 23:00:00+00:00  165.449997           1299.0  208.199997   
2020-02-29 23:00:00+00:00  161.899994           1299.0  175.300003   
2020-03-31 22:00:00+00:00  160.449997           1299.0  187.500000   
2020-04-30 22:00:00+00:00  171.350006           1299.0  185.800003   
2020-05-31 22:00:00+00:00  160.100006           1299.0  209.000000   
2020-06-30 22:00:00+00:00  154.800003           1299.0  218.500000   
2020-07-31 22:00:00+00:00  171.350006           1299.0  220.199997   
2020-08-31 22:00:00+00:00  167.149994           1299.0  229.199997   
2020-09-30 22:00:00+00:00  173.399994           1299.0  216.399994   
2020-10-31 23:00:00+00:00  169.199997           1299.0  226.000000   
2020-11-30 23:00:00+00:00  166.000000           1299.0  229.000000   
2020-12-31 23:00:00+